In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-235075
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-235075


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "projectCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, using it")
except ComputeTargetException:
    print("Creating a new compute cluster")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())




Found existing cluster, using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-06-05T12:48:07.534000+00:00', 'errors': None, 'creationTime': '2023-06-05T12:48:02.925909+00:00', 'modifiedTime': '2023-06-05T12:48:06.724600+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
    '--max_iter': choice(50,150,300, 500)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# # Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# # Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = "./", compute_target=compute_target, script='train.py', environment=sklearn_env)

# Create a SKLearn Estimator to specify the configuration details of your training job
#est = SKLearn(source_directory = "./", compute_target=compute_target, entry_script='train.py')

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,primary_metric_name='Accuracy',primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, hyperparameter_sampling=ps, policy=policy, max_total_runs = 20)

In [16]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9
Web View: https://ml.azure.com/runs/HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-235075/workspaces/quick-starts-ws-235075&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-05T14:01:43.732204][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space
[2023-06-05T14:01:44.4665805Z][SCHEDULER][INFO]Scheduling job, id='HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_0' 
[2023-06-05T14:01:44.6993229Z][SCHEDULER][INFO]Scheduling job, id='HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_1' 
[2023-06-05T14:01:44.7022722Z][SCHEDULER][INFO]Scheduling job, id='HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_2' 
[2023-06-05T14:01:44.7153712Z][SCHEDULER][INFO]Scheduling job, id='HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_3' 
[2023-06-05T14:01:44.8114867Z][SCHEDULER][INFO]Scheduling job, id='HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_4' 
[2023-06-0

{'runId': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9',
 'target': 'projectCluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-05T14:01:43.022616Z',
 'endTimeUtc': '2023-06-05T14:06:45.867054Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3d68931d-a52f-4656-8e15-e847ad8bebed',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '28',
  'score': '0.9061665452779801',
  'best_child_run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_12',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_12'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCl

In [17]:
import joblib
# Get your best run and save the model from that run.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))


best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())
print("Best run file names :", best_run.get_file_names())



[{'run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_12', 'hyperparameters': '{"--C": 100, "--max_iter": 300}', 'best_primary_metric': 0.9061665452779801, 'status': 'Completed'}, {'run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_13', 'hyperparameters': '{"--C": 10, "--max_iter": 300}', 'best_primary_metric': 0.9060451565914057, 'status': 'Completed'}, {'run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_2', 'hyperparameters': '{"--C": 100, "--max_iter": 150}', 'best_primary_metric': 0.9060451565914057, 'status': 'Completed'}, {'run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_5', 'hyperparameters': '{"--C": 0.01, "--max_iter": 500}', 'best_primary_metric': 0.9058023792182569, 'status': 'Completed'}, {'run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_9', 'hyperparameters': '{"--C": 0.1, "--max_iter": 300}', 'best_primary_metric': 0.9056809905316825, 'status': 'Completed'}, {'run_id': 'HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_14', 'hyperparameters': '{"--C": 1, "--max_iter": 500}', 'be

In [18]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_4fb5ee63-0f84-4ef9-a2a6-7bd85a906ab9_12,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [11]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_96bd1915-f5e3-44fd-8bba-ed84c0c31ec1,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_96bd1915-f5e3-44fd-8bba-ed84c0c31ec1',
 'target': 'projectCluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-05T13:03:31.400895Z',
 'endTimeUtc': '2023-06-05T13:43:17.967405Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'projectCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-235075","workspace_name":"quick-starts-ws-235075","region":"southcentralus","compute_target":"projectCluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_

In [12]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()

print(best_run)
print(fitted_model)

print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())




Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Run(Experiment: udacity-project,
Id: AutoML_96bd1915-f5e3-44fd-8bba-ed84c0c31ec1_27,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('3', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('xgboostclassifier

In [13]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_96bd1915-f5e3-44fd-8bba-ed84c0c31ec1_27,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#compute_target.delete()